<a href="https://colab.research.google.com/github/rupeshthapa123/NotebookProject/blob/main/Rupesh_Thapa_Lab02_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab02. Transfer Learning

In this lab, you will learn to train a pretrained model to decide whether a movie review is positive or negative.

Transfer learning allows developers to repurpose trained models and use a different dataset to retrain models, avoiding the need to start retraining from scratch. This way, previous knowledge inside of a model is exported into the new model directly, this improving accuracy and performance.

Once completed, submit your ipynb file to the dropbox in D2L.

1. Implement an initial text classifier without transfer learning by just training a simple classifier. (30 marks)

In [ ]:
# Import necessary libraries and modules from the FastAI library
from fastai.text.all import *
# Untar the data from the URL specified and store it in the path variable
path = untar_data(URLs.IMDB)
# Create a TextDataLoaders object using the path variable and a validation set with the 'test' keyword
dls = TextDataLoaders.from_folder(path, valid='test')
# Display a batch of data from the dataset
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * ! ! - xxup spoilers - ! ! * \n\n xxmaj before i begin this , let me say that i have had both the advantages of seeing this movie on the big screen and of having seen the "" authorized xxmaj version "" of this movie , remade by xxmaj stephen xxmaj king , himself , in 1997 . \n\n xxmaj both advantages made me appreciate this version of "" the xxmaj shining , "" all the more . \n\n xxmaj also , let me say that xxmaj i 've read xxmaj mr . xxmaj king 's book , "" the xxmaj shining "" on many occasions over the years , and while i love the book and am a huge fan of his work , xxmaj stanley xxmaj kubrick 's retelling of this story is far more compelling … and xxup scary . \n\n xxmaj kubrick",pos
2,"xxbos xxmaj titanic directed by xxmaj james xxmaj cameron presents a fictional love story on the historical setting of the xxmaj titanic . xxmaj the plot is simple , xxunk , or not for those who love plots that twist and turn and keep you in suspense . xxmaj the end of the movie can be figured out within minutes of the start of the film , but the love story is an interesting one , however . xxmaj kate xxmaj winslett is wonderful as xxmaj rose , an aristocratic young lady betrothed by xxmaj cal ( billy xxmaj zane ) . xxmaj early on the voyage xxmaj rose meets xxmaj jack ( leonardo dicaprio ) , a lower class artist on his way to xxmaj america after winning his ticket aboard xxmaj titanic in a poker game . xxmaj if he wants something , he goes and gets it",pos
3,"xxbos xxmaj some have praised _ xxunk _ as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think _ the xxmaj dirty xxmaj dozen _ ) but _ atlantis _ represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before",neg
4,"xxbos xxmaj here are the matches . . . ( adv . = advantage ) \n\n xxmaj the xxmaj warriors ( ultimate xxmaj warrior , xxmaj texas xxmaj tornado and xxmaj legion of xxmaj doom ) v xxmaj the xxmaj perfect xxmaj team ( mr xxmaj perfect , xxmaj ax , xxmaj smash and xxmaj crush of xxmaj demolition ): xxmaj ax is the first to go in seconds when xxmaj warrior splashes him for the pin ( 4 - 3 adv . xxmaj warriors ) . i knew xxmaj ax was n't a healthy man but if he was that unhealthy why bother have him on the card ? xxmaj this would be his last xxup ppv . xxmaj eventually , both xxmaj legion of xxmaj doom and xxmaj demolition job out cheaply via double disqualification ( 2 - 1 adv . xxmaj warriors ) . xxmaj perfect applies",neg
5,"xxbos xxmaj the premise of this movie has been tickling my imagination for quite some time now . xxmaj we 've all heard or read about it in some kind of con - text . xxmaj what would you do if you wer

In [ ]:
#Create a text classifier learner object using the pre-trained AWD_LSTM model, dls (the dataset loader),
#and a dropout multiplier of 0.5. Set the accuracy as the evaluation metric.
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult = 0.5, metrics = accuracy)

In [ ]:
#This line of code calls the fine_tune method of the learn object, passing in two arguments:
#the number of epochs (4) and the learning rate (1e-2). This method will fine-tune the model based
#on the training data, using the Adam optimizer and a loss function specified by the learner.
learn.fine_tune(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.473817,0.396800,0.819040,03:20


epoch,train_loss,valid_loss,accuracy,time
0,0.281579,0.231140,0.909160,07:06
1,0.217705,0.198485,0.922640,07:06


In [ ]:
# Use the learn model to predict the sentiment of the text "That movie was not so great!"
learn.predict("That movie was not so great!")

('neg', tensor(0), tensor([0.5105, 0.4895]))

In [ ]:
# Display the results of the last training or validation run
learn.show_results()

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,"xxbos ( some spoilers included :) \n\n xxmaj although , many commentators have called this film surreal , the term fits poorly here . xxmaj to quote from xxmaj encyclopedia xxmaj xxunk 's , surreal means : \n\n "" fantastic or incongruous imagery "" : xxmaj one need n't explain to the unimaginative how many ways a plucky ten - year - old boy at large and seeking his fortune in the driver 's seat of a red xxmaj mustang could be fantastic : those curious might read xxmaj james xxmaj kincaid ; but if you asked said lad how he were incongruous behind the wheel of a sports car , he 'd surely protest , "" no way ! "" xxmaj what fantasies and incongruities the film offers mostly appear within the first fifteen minutes . xxmaj thereafter we get more iterations of the same , in an ever",pos,neg
2,"xxbos i was recently given this film on xxup dvd as a gift , and was unsure at first if it would appeal ( although one of my favourite actors has a leading role ) . xxmaj in fact , it 's on its way to becoming a favourite . \n\n xxmaj first of all : thankfully , it 's * not * the same as the book , the ending of which i think is excessively melodramatic . xxmaj secondly : it 's one of the best films xxmaj i 've seen about the xxmaj first xxmaj world xxmaj war . "" what ? "" you may ask . "" it 's not a war film ! "" xxmaj true : we see no battles or xxunk , no trenches , no gas . xxmaj but it shows the cost of war , the damage done to the lives",pos,pos
3,"xxbos "" the belief in the xxmaj big xxmaj other as an invisible power structure which exists in the xxmaj real is the most succinct definition of paranoia . ""  xxmaj slavoj xxmaj zizek \n\n xxmaj this is a review of "" marathon xxmaj man "" and "" the xxmaj falcon and the xxmaj snowman "" , two films by director xxmaj john xxmaj schlesinger . \n\n xxmaj though xxmaj hitchcock and xxmaj lang brought the "" conspiracy thriller "" to xxmaj hollywood , the genre only blossomed in the late 60s and 70s , with films like "" the xxmaj parallax xxmaj view "" , "" z "" , "" marathon xxmaj man "" , "" capricorn xxmaj one "" , "" the xxmaj manchurian xxmaj candidate "" , "" three xxmaj days of the xxmaj condor "" and "" all xxmaj the xxmaj president 's xxmaj men",pos,pos
4,"xxbos xxmaj my qualifications for this review ? i own all the xxmaj alien and xxmaj predator movies & i have and have read almost all the books i can find that are related to this series . \n\n i can safely say , this movie is a xxmaj stinker . xxmaj save your money & do n't waste your time . xxmaj if you like mindless action , mindless gore , no plot to speak of & like being taken by xxmaj hollywood , see the movie . \n\n xxmaj if you are a serious xxmaj alien series fan , send a message to the over stuffed , over paid suits in xxmaj hollywood & 20th xxmaj century xxmaj fox & do n't give them a penny . \n\n xxmaj this movie has so many plot holes in it you could sieve pasta through it . xxmaj read the",neg,neg
5,"xxbos xxmaj this is a simple episode ad so far after watching all of the xxmaj season 11 episodes ( with the exception of the xxmaj xxunk trilogy ) this is the one that made laugh the most , definitely is my favourite so far of xxmaj season 11 . xxmaj so basically xxmaj cartman

2. Use ULMFiT technique and implement transfer learning to export knowledge from the Wikipedia model to the model trained using the IMDB dataset and then train a final classifier. (70 marks)

In [ ]:
# Create a dataloader for language modeling from the specified folder
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)
# Show a batch of data from the dataloader
dls_lm.show_batch(max_n=5)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


,text,text_
0,"xxbos xxmaj while lifting various plot elements from the original ( nasty virus , xxmaj wildfire containment facility , odd - man - out auto - destruct xxunk , etc ) we get a whole pile of additional elements thrown in , and for no apparent good reason . \n\n xxmaj ben xxmaj bratt 's character has a substance abusing wife and an estranged son … this goes no where . \n\n","xxmaj while lifting various plot elements from the original ( nasty virus , xxmaj wildfire containment facility , odd - man - out auto - destruct xxunk , etc ) we get a whole pile of additional elements thrown in , and for no apparent good reason . \n\n xxmaj ben xxmaj bratt 's character has a substance abusing wife and an estranged son … this goes no where . \n\n xxmaj"
1,"the xxunk cast looks so much older , but you must bear in mind that these are people who have been on xxmaj broadway and worked their tails off to be there . xxmaj if the entire cast was green and fresh then i believe that would have compromised the casting process by which producers choose their talent . xxbos well , xxmaj i 'm a xxup princess xxup hours ultimate fan","xxunk cast looks so much older , but you must bear in mind that these are people who have been on xxmaj broadway and worked their tails off to be there . xxmaj if the entire cast was green and fresh then i believe that would have compromised the casting process by which producers choose their talent . xxbos well , xxmaj i 'm a xxup princess xxup hours ultimate fan …"
2,"xxmaj sang , xxmaj les xxmaj amants du xxmaj pont xxmaj neuf . xxmaj in fact those films were not that great but it has the violence of youth , the beauty of juvenile wilderness . xxmaj carax in these three movies was well aware of what cinema was , but he tried to make his own vision of the art , without thinking about about all he have seen , but","sang , xxmaj les xxmaj amants du xxmaj pont xxmaj neuf . xxmaj in fact those films were not that great but it has the violence of youth , the beauty of juvenile wilderness . xxmaj carax in these three movies was well aware of what cinema was , but he tried to make his own vision of the art , without thinking about about all he have seen , but using"
3,"i can think of , which then is far weaker in every respect , is the xxmaj royal xxmaj tenenbaums . \n\n xxmaj four xxmaj shades of xxmaj brown , as the title would be in xxmaj english , tells the story about a stressed out xxunk and his wife , and his elderly parents ( who are traveling magicians ) ; about the receptionist at an animal crematory and his family","can think of , which then is far weaker in every respect , is the xxmaj royal xxmaj tenenbaums . \n\n xxmaj four xxmaj shades of xxmaj brown , as the title would be in xxmaj english , tells the story about a stressed out xxunk and his wife , and his elderly parents ( who are traveling magicians ) ; about the receptionist at an animal crematory and his family misadventures"
4,"would be that some of the characters are pushed to the background , never getting a chance to fully shine , but a film this good is a rare thing . xxmaj see it now you wo n't regret it . 9 / 10 xxbos i can not believe i sat through this utter waste of time . i was just too fascinated by how unspeakably bad it was that i could","be that some of the characters are pushed to the background , never getting a chance to fully shine , but a film this good is a rare thing . xxmaj see it now you wo n't regret it . 9 / 10 xxbos i can not believe i sat through this utter waste of time . i was just too fascinated by how unspeakably bad it was that i could n't"


In [ ]:
# Create a language model learner using the AWD_LSTM architecture
learn = language_model_learner( dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [ ]:
# Train the language model for one epoch using the one-cycle policy with a learning rate of 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.398916,4.107900,0.286021,60.818874,24:08


In [ ]:
# Save the learner's state after one epoch of training
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
# Load the model from the specified epoch
learn = learn.load('1epoch')

In [ ]:
# Unfreeze all layers in the model
learn.unfreeze()
# Train the model for 1 epochs with a learning rate of 1e-3
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.022898,3.831999,0.312557,46.154713,25:55


In [ ]:
# Save the encoder of the learning model, named 'finetuned'
learn.save_encoder('finetuned')

In [ ]:
# Define a string variable TEXT with a message
TEXT = "I liked this movie because"
# Define a integer variable N_WORDS with the number of words to generate
N_WORDS = 40
# Define a integer variable N_SENTENCES with the number of sentences to generate
N_SENTENCES = 2
# Create a list of predictions using the learn object and the TEXT, N_WORDS, and N_SENTENCES variables
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
        for _ in range(N_SENTENCES)]

# Print the list of predictions joined by newline characters
print("\n".join(preds))

i liked this movie because of my family knowing that it was a flashback , but i could n't even sleep . Of course , was a child , i think i must say because it was better than i expected . It
i liked this movie because of the fact that it was made in a VERY different way than you can see . The actors are excellent . The costumes are uniformly good . The cinematography is just terrible . It


In [ ]:
# Create a TextDataLoaders object from a folder containing the IMDB dataset, with the validation set set to 'test', and using the vocabulary from the language model data loader
dls_class = TextDataLoaders.from_folder(
    # Untar the data from the URL
    untar_data(URLs.IMDB),
    # Set the validation set to 'test'
    valid='test',
    # Use the vocabulary from the language model data loader
    text_vocab=dls_lm.vocab)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
# Create a text classifier learner object using the dataloader, the AWD_LSTM model, a dropout multiplier of 0.5, and accuracy as the metric
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
# Load the encoder 'finetuned'
learn = learn.load_encoder('finetuned')

In [ ]:
# This line of code fits the model on one cycle with a learning rate of 2e-2
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.271397,0.232509,0.903520,03:33


In [ ]:
# Freeze the first -2 layers of the model
learn.freeze_to(-2)
# Train the model for 1 epoch using a learning rate of 1e-2
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.236233,0.189057,0.926040,03:58


In [ ]:
# Freeze the first 3 layers of the model
learn.freeze_to(-3)
# Train the model for 1 epoch using a learning rate of 5e-3 and a slice of the learning rate range between 5e-3/(2.6**4) and 5e-3
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.207741,0.168618,0.936400,05:40


In [ ]:
# Unfreeze all layers in the model
learn.unfreeze()
# Train the model for 2 epochs with a learning rate of 1e-3
learn.fit_one_cycle(1, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.156328,0.162547,0.938600,07:07


## COMMENT

As, the resources i.e (2 hours and 50 minutes ) provided by Google Colab for Runtime TPU was not enough to run all the epochs so, I reduced every epoch to 1 and ran the model except for the first fine tune method which was changed to 2 epoch. Each epoch takes about 25 minutes when ran with full resources available (TPU).  

Accuracy could have been better with high number of epochs as the accuracy was around 0.2 to 0.3 for first two model methods but for last few model it was around 0.92 which is great.